In [ ]:
import pandas as pd
import numpy as np
import os
import platform

In [ ]:
"""CONFIGURATION VARIABLES"""
rows_to_be_dropped = ["Conversion", "Conversion " ,"Marketing Input", "TV - Home", "GRPs estimated for News Dark Period" ]
columns_to_be_dropped = ["comments"]

fiction_1 = pd.read_excel("/content/Colors Data - For Analysis 6th June update (1).xlsx", sheet_name=0,  skiprows=2, dtype="str")
# Clean column names
fiction_1.columns = fiction_1.columns.str.lower().str.replace("-", "").str.replace(" ", "_")
fiction_1 = fiction_1.loc[:, ~fiction_1.columns.str.startswith("unnamed")]
fiction_1 = fiction_1.drop(columns = columns_to_be_dropped)

# Rename column 'shows' to 'metric'
fiction_1.rename(columns={"shows": "metric"}, inplace=True)
fiction_1['metric'] = fiction_1['metric'].str.strip()

fiction_1.reset_index(drop=True, inplace=True)

# drop irrelevant
fiction_1 = fiction_1[~fiction_1['metric'].isin(rows_to_be_dropped)].dropna(how="all")

In [ ]:
fiction_1.reset_index(drop= True, inplace = True)
# fiction_1

In [ ]:
# new_columns =   list(fiction_1["metric"])
# new_columns.insert(0, "shows")

# # making the new dataframe to chnge the rows
# main_data = pd.DataFrame(columns = new_columns)

# for i in range(len(new_columns)) :
#   if i == 0 :
#     data = list(fiction_1.columns)[1:]
#     main_data[new_columns[i]] = data
#   else :
#     data = fiction_1.iloc[i-1, 1:].values
#     main_data[new_columns[i]] = list(data)

# main_data.columns = main_data.columns.str.lower().str.replace("-", "").str.replace(" ", "_")

In [ ]:
markers = ["Launch Performance", "Conversion Launch Day", "Conversion Launch Week", "Star Plus Slot Reach% (Pre 4 Wks)",  "Campaign GRPs (Home)", "TV - NW Spends", "TV - OSN Spends", "Digital",  "Social Video Spends (YT, FB, Insta, Sharechat, etc)", "OTT Spends (Hotstar, Zee5, etc.)", "GDN/Programmatic/Xasis Spends", "Sub-mediums", "Other Spends (Glance, Zapr, CTV, etc.)", "Overall Digital  Spends", "Spends on Influencers & Paid Reviews - CAT A", "Other Marketing Spends", "Owned & Earned Social Media", "Earned Media - PR", "Cross Network GRPs Broken By Genre till Launch Week", "OSN GRPs Broken By Genre till Launch Week", "Previous slot Daily Avg Reach % in Launch week"]
data_groups = []

for ind in range(len(markers)):
    if ind==0:
        start_index = 0
        end_entry = markers[ind]
        end_index = fiction_1.index[fiction_1['metric'] == end_entry][0]-1
        data = fiction_1.loc[start_index:end_index]
        data_groups.append(data)
        start_entry = markers[ind]
        end_entry = markers[ind+1]
        start_index = fiction_1.index[fiction_1['metric'] == start_entry][0]
        end_index = fiction_1.index[fiction_1['metric'] == end_entry][0]-1
        data = fiction_1.loc[start_index:end_index]
        data_groups.append(data)
    elif ((ind>0) & (ind<=len(markers)-2)):
        start_entry = markers[ind]
        end_entry = markers[ind+1]
        start_index = fiction_1.index[fiction_1['metric'] == start_entry][0]
        end_index = fiction_1.index[fiction_1['metric'] == end_entry][0]-1
        data = fiction_1.loc[start_index:end_index]
        data_groups.append(data)
    else:
        start_entry = markers[ind]
        start_index = fiction_1.index[fiction_1['metric'] == start_entry][0]
        end_index = len(fiction_1)+1
        data = fiction_1.loc[start_index:end_index]
        data_groups.append(data)

data_long = []

for i in range(len(data_groups)):
    df = pd.DataFrame(data_groups[i])
    df = df.loc[:, df.columns.notna()]
    df2  = df.T.reset_index(drop=True)
    df2.columns = df2.iloc[0]
    df2 = df2.drop(0)
    df2["shows"] = list(df.columns)[1:]
    df2 = df2.dropna(axis = 1, how=   "all")
    df2.reset_index(drop=True, inplace=True)
    columns = ["shows"] + [col for col in df2.columns if col != "shows"]
    main_df = df2[columns]
    main_df.columns = main_df.columns.str.lower().str.replace("-", "").str.replace(" ", "_")
    data_long.append(main_df)


show_details, launch_perf, conversion_ld, conversion_lw, comp_chan_rch, tv_home_promo, tv_nw_promo, tv_osn_promo, digi_tg, social_media, ott, gdn_prog, sub_mediums, others_glance_zapr, overall_digi_spends, influencers, dth_ooh_oga_cin_celeb, social_own_earn, pr_earned_new_metrics, nw_grp_split, osn_grp_split, pre_slot_rch = data_long[0], data_long[1], data_long[2], data_long[3], data_long[4], data_long[5], data_long[6], data_long[7], data_long[8], data_long[9], data_long[10], data_long[11], data_long[12], data_long[13], data_long[14], data_long[15], data_long[16], data_long[17], data_long[18], data_long[19], data_long[20], data_long[21]

In [ ]:
def convert_date(date_str):
  try:
    if "'" in date_str:
      parts = date_str.split('-')
      day = parts[0]
      month = parts[1][:3]
      year = '20' + parts[1][4:]
      new_date_str = f"{day}-{month}-{year}"
      return pd.to_datetime(new_date_str, format='%d-%b-%Y')
    else:
      return pd.to_datetime(date_str)
  except Exception as e:
      return pd.NaT

In [ ]:
## show details

show_details['launch_date'] = show_details['launch_date'].apply(convert_date)
show_details['year'] = show_details['launch_date'].dt.year
show_details['month'] = show_details['launch_date'].dt.month
show_details['day'] = show_details['launch_date'].dt.day


show_details = show_details.rename(columns={'mf/ss': 'mf_ss'})
show_details['slot_time'] = show_details['slot_time'].astype('category')
show_details['mf_ss'] = show_details['mf_ss'].astype('category')
show_details['genre'] = show_details['genre'].astype('category')
show_details['prominent_leads_(if_any)'] = show_details['prominent_leads_(if_any)'].apply(lambda x: 'yes' if pd.notnull(x) else 'no').astype("category")
show_details['celebrity_endorsers_(if_any)'] = show_details['celebrity_endorsers_(if_any)'].apply(lambda x: 'yes' if pd.notnull(x) else 'no').astype("category")
show_details['regional_remake_(pls_specify)'] = show_details['regional_remake_(pls_specify)'].apply(lambda x: 'yes' if pd.notnull(x) else 'no').astype("category")

In [ ]:
## Launch Performance

def time_to_seconds(time_str):
  if pd.notna(time_str):
    time_parts = time_str.split(':')
    hours = int(time_parts[0])
    minutes = int(time_parts[1])
    seconds = float(time_parts[2])
    return hours * 3600 + minutes * 60 + seconds
  else:
    return np.nan

def format_time(seconds):
  if pd.notna(seconds) :
    mins = seconds // 60
    secs = seconds % 60
    return f"{int(mins)}.{int(secs)}"
  else:
    return np.nan

launch_perf_time_col = ["pre_4_wks_tsv", "ld_tsv"	, "lw_tsv"]
for col in launch_perf_time_col :
    launch_perf[col] = launch_perf[col].apply(time_to_seconds)
    launch_perf[col] = launch_perf[col].apply(format_time)

launch_perf = launch_perf.replace("-", "0")
launch_perf = launch_perf.apply(pd.to_numeric, errors = "ignore")

In [ ]:
## conversion Launch Day
conversion_ld_col =  [col+"(till_ld)" for col in conversion_ld.columns if col != "shows"]
conversion_ld.columns = ["shows"] + conversion_ld_col

for col in conversion_ld_col :
    conversion_ld[col] = conversion_ld[col].apply(pd.to_numeric, errors = 'coerce')

In [ ]:
## conversion Launch Week
conversion_lw_col =  [col+"(till_lw)" for col in conversion_lw.columns if col != "shows"]
conversion_lw.columns = ["shows"] + conversion_lw_col

for col in conversion_lw_col :
    conversion_lw[col] = conversion_lw[col].apply(pd.to_numeric, errors = 'coerce')



## competition and channel reach
comp_chan_rch = comp_chan_rch.apply(pd.to_numeric, errors = 'ignore')


## TV home promo
tv_home_promo = tv_home_promo.apply(pd.to_numeric, errors = 'ignore')



## TV nw promo

tv_nw_promo = tv_nw_promo.replace("-", np.nan)
temp_nw= tv_nw_promo["regional_market_input_(pls_specify_markets_if_any)"]
reg_names_tv_nw = pd.Series(temp_nw).str.split(',').explode().str.strip().unique().tolist()

reg_market_tv_nw = np.zeros((len(tv_nw_promo), len(reg_names_tv_nw)))
for k, markets_str in enumerate(temp_nw):
    if pd.notna(markets_str):
        markets = [m.strip() for m in markets_str.split(',')]
        indices = [reg_names_tv_nw.index(m) for m in markets if m in reg_names_tv_nw]
        reg_market_tv_nw[k, indices] = 1

reg_market_tv_nw_df = pd.DataFrame(reg_market_tv_nw, columns=[f'{name} (nw)' for name in reg_names_tv_nw])
reg_market_tv_nw_df = reg_market_tv_nw_df.drop(columns= "nan (nw)")

tv_nw_promo_without_rm = tv_nw_promo.drop(columns=["regional_market_input_(pls_specify_markets_if_any)"], axis = 1)
tv_nw_promo = pd.concat([tv_nw_promo_without_rm, reg_market_tv_nw_df], axis=1)
tv_nw_promo = tv_nw_promo.apply(pd.to_numeric, errors = 'ignore')



## tv osn promo
tv_osn_promo = tv_osn_promo.replace("-", np.nan)
temp_osn = tv_osn_promo["regional_market_input_(pls_specify_markets_if_any)"]
regional_market_osn = pd.Series(temp_osn).str.split(',').explode().str.strip().unique().tolist()
reg_mar_tv_osn = np.zeros((len(tv_osn_promo), len(regional_market_osn)))

for k, markets_str in enumerate(temp_osn):
    if pd.notna(markets_str):
        markets = [m.strip() for m in markets_str.split(',')]
        indices = [regional_market_osn.index(m) for m in markets if m in regional_market_osn]
        reg_mar_tv_osn[k, indices] = 1

reg_mar_tv_osn_df = pd.DataFrame(reg_mar_tv_osn, columns=[f'{name} (osn)' for name in regional_market_osn])
reg_mar_tv_osn_df = reg_mar_tv_osn_df.drop(columns= "nan (osn)")

tv_osn_promo_without_rm = tv_osn_promo.drop(columns=["regional_market_input_(pls_specify_markets_if_any)"], axis = 1)
tv_osn_promo = pd.concat([tv_osn_promo_without_rm, reg_mar_tv_osn_df], axis=1)
tv_osn_promo = tv_osn_promo.apply(pd.to_numeric, errors = 'ignore')



## digital

temp_dg = digi_tg["markets"]
markets = pd.Series(temp_dg).str.split(',').explode().str.strip().unique().tolist()
digi_mar = pd.get_dummies(temp_dg.str.split(',').explode().str.strip()).groupby(level=0).sum()

digi_mar.columns = [col+"(digi)" for col in digi_mar.columns]
digi_market_col = [col for col in digi_mar.columns if not any(char in col for char in '&\n:')]
digi_mar1 = digi_mar[digi_market_col]

digi_tg = pd.concat([digi_tg.drop(columns=["markets"]), digi_mar1], axis=1)
digi_tg = digi_tg.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
##social
social_media_cc =  [col+"(social video)" for col in social_media.columns if col not in ["shows", "social_video_spends_(yt,_fb,_insta,_sharechat,_etc)"]]
social_media.columns = ["shows", "social_video_spends_(yt,_fb,_insta,_sharechat,_etc)"] + social_media_cc
social_media = social_media.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## ott
ott_cc =  [col+"(ott)" for col in ott.columns if col not in ["shows", "ott_spends_(hotstar,_zee5,_etc.)"]]
ott.columns = ["shows", "ott_spends_(hotstar,_zee5,_etc.)"] + ott_cc
ott = ott.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## gdn
gdn_prog_cc =  ["impressions(GDN)", "views(GDN)", "impact_spends_(ld_page_takeover)"]
gdn_prog.columns = ["shows", "gdn/programmatic/xasis_spends	"] + gdn_prog_cc
gdn_prog = gdn_prog.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## sub_mediums

sub_mediums_cc = ["impressions(sub_mediums)", "views(sub_mediums)"]
sub_mediums.columns = ["shows", "sub_mediums"] + sub_mediums_cc
sub_mediums = sub_mediums.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## others_glance_zapr

others_glance_zapr_cc = [col+"(oth_glance_zapr)" for col in others_glance_zapr.columns if col not in ["shows", "other_spends_(glance,_zapr,_ctv,_etc.)"]]
others_glance_zapr.columns = ["shows", "other_spends_(glance,_zapr,_ctv,_etc.)" ] + others_glance_zapr_cc
others_glance_zapr = others_glance_zapr.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## overall_digi_spends
overall_digi_spends = overall_digi_spends.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## influencers
influencers = influencers.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## dth_ooh_oga_cin_celeb
dth_ooh_oga_cin_celeb = dth_ooh_oga_cin_celeb.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## social_own_earn
social_own_earn = social_own_earn.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.name != 'shows' else col)

In [ ]:
## pr_earned_new_metrics
def extract_numeric(value):
    numeric_part = pd.to_numeric(value.str.extract(r'([0-9]*\.?[0-9]+)')[0], errors='coerce')
    return numeric_part

def replaced_genre(text):
    if isinstance(text, str):
        if "-" in text:
            return text.split("-", 1)[1].strip()
        elif "/" in text:
            return text.split("/")[1].strip()
    return text


pr_column = ["total_pr_value_(from_prebuzz_to_launch)", "total_reach_(from_prebuzz_to_launch)"]
pr_earned_new_metrics[pr_column] = pr_earned_new_metrics[pr_column].apply(extract_numeric)
pr_earned_new_metrics["replaced_genre"] = pr_earned_new_metrics["which_show_&_genre_did_the_launch_show_replace"].apply(replaced_genre)
pr_earned_new_metrics = pr_earned_new_metrics.drop(columns=["which_show_&_genre_did_the_launch_show_replace"])
pr_earned_new_metrics = pr_earned_new_metrics.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## nw_grp_split

nw_grp_split_cc = [col+"(nw)" for col in nw_grp_split.columns if col not in ["shows"]]
nw_grp_split.columns = ["shows"] + nw_grp_split_cc
nw_grp_split = nw_grp_split.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## osn_grp_split
osn_grp_split_cc = [col+"(osn)" for col in osn_grp_split.columns if col not in ["shows"]]
osn_grp_split.columns = ["shows"] + osn_grp_split_cc
osn_grp_split = osn_grp_split.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## pre_slot_rch# Remove columns that are entirely NaN
pre_slot_rch = pre_slot_rch.loc[:, pre_slot_rch.columns.notna()]
pre_slot_rch = pre_slot_rch.apply(pd.to_numeric, errors = 'ignore')

In [ ]:
## combine the all dataframes to get D0
show_dataframes = [show_details, launch_perf, conversion_ld, conversion_lw, comp_chan_rch, tv_home_promo, tv_nw_promo, tv_osn_promo, digi_tg, social_media, ott, gdn_prog, sub_mediums, others_glance_zapr, overall_digi_spends, influencers, dth_ooh_oga_cin_celeb, social_own_earn, pr_earned_new_metrics, nw_grp_split, osn_grp_split, pre_slot_rch]

combine_df = show_dataframes[0]
for df in show_dataframes[1:]:
    combine_df = combine_df.merge(df, on='shows', how='left')

In [ ]:
combine_df.to_csv("D0_colors_data.csv")